In [ ]:
## Show case of flight data status change

In [ ]:
def read_ccloud_config(config_file):
    omitted_fields = set(['schema.registry.url', 'basic.auth.credentials.source', 'basic.auth.user.info'])
    conf = {}
    with open(config_file) as fh:
        for line in fh:
            line = line.strip()
            if len(line) != 0 and line[0] != "#":
                parameter, value = line.strip().split('=', 1)
                if parameter not in omitted_fields:
                    conf[parameter] = value.strip()
    return conf

In [ ]:
from confluent_kafka import Producer
import json
# value ={"flight_num": "AA423", "from":"JFK", "to":"LAX", "old_departure_time": "NA", "old_arrival_time":"NA","departure_time": "2023-09-17T23:00:00", "arrival_time":"2023-09-18T01:00:00", 
#         "reason":"First time run","flight_status": "ontime"}


value ={"flight_num": "AA423", "from":"JFK", "to":"LAX", "old_departure_time": "2023-09-17T23:00:00", 
        "old_arrival_time":"2023-09-18T01:00:00","departure_time": "2020-09-18T02:00:00", 
        "arrival_time":"2023-09-18T04:00:00", "reason":"Forecasted bad weather at destination",
        "flight_status": "delayed"}


topic = "flight_status"
value = json.dumps(value)
producer = Producer(read_ccloud_config("client.properties"))
producer.produce(topic, key="key", value=value)
producer.flush()


In [ ]:

from confluent_kafka import Producer
import json
new_policy = {"id": "e22891ad-5850-42ac-b856-270615ab1f3a",
"policy_type": "checked baggage policy",
"policy": """American Airlines' checked baggage policy states that passengers are allowed to check up to 2 bags on their flights. However, charges will apply for all checked bags except for First Class and Business Class passengers, AAdvantage Executive Platinum, Platinum Pro, and Platinum members, and Oneworld Emerald members. 

For all other passengers, the first checked bag will cost $30 for domestic flights and flights to/from Canada, Mexico, Central America, and the Caribbean, and $60 for transatlantic flights. The second bag will cost $40 for domestic flights and flights to/from Canada, Mexico, Central America, and the Caribbean, and $100 for transatlantic flights. 

Additional checked bags will each cost $250 for all flights except for flights to/from Brazil, where each additional checked bag costs $85. Oversized and overweight baggage fees also apply, with fees ranging from $100 to $450 per bag depending on the weight and size. 

Passengers are encouraged to check their baggage allowance and fees prior to their flights to avoid any unexpected charges at the airport.""",
"airline":  "American Airlines",
"active":  "true"}
topic = "general"
value = json.dumps(new_policy)
producer = Producer(read_ccloud_config("client.properties"))
producer.produce(topic, key="key", value=value)
producer.flush()


In [ ]:
from pathlib import Path  
import os
import openai
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from openai.embeddings_utils import get_embedding, cosine_similarity
import inspect
env_path = Path('.') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
openai.api_key =  os.environ.get("AZURE_OPENAI_API_KEY")
openai.api_base =  os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_SEARCH_SERVICE_ENDPOINT=os.environ.get("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME=os.environ.get("AZURE_SEARCH_INDEX_NAME")
AZURE_SEARCH_ADMIN_KEY=os.environ.get("AZURE_SEARCH_ADMIN_KEY")

openai.api_type = "azure"
emb_engine = os.getenv("AZURE_OPENAI_EMB_DEPLOYMENT")
emb_engine = emb_engine.strip('"')



In [ ]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.models import Vector 
import openai


credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
azcs_search_client = SearchClient(AZURE_SEARCH_SERVICE_ENDPOINT, index_name =AZURE_SEARCH_INDEX_NAME , credential=credential)
def generate_embeddings(text):
    print("emb_engine", emb_engine)
    openai.api_version = "2023-05-15"
    response = openai.Embedding.create(
        input=text, engine=emb_engine)
    embeddings = response['data'][0]['embedding']
    return embeddings

def search_knowledgebase(search_query):
    vector = Vector(value=generate_embeddings(search_query), k=3, fields="embedding")
    print("search query: ", search_query)
    results = azcs_search_client.search(  
        search_text=search_query,  
        vectors= [vector],
        select=["sourcepage","content"],
        top=5
    )  
    text_content =""
    for result in results:  
        text_content += f"{result['sourcepage']}\n{result['content']}\n"
    print("text_content", text_content)
    return text_content

def add_or_update(new_flight):
    azcs_search_client.merge_or_upload_documents(documents = [new_flight])



In [ ]:
# Semantic Hybrid Search
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.models import Vector 
import openai
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)  


query = "American Airlines baggage limit"

vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  
filter = "airline eq 'American Airlines'"
results = azcs_search_client.search(  
    search_text=query,  
    vectors=[vector],
    # filter=filter,
    select=["id", "policy_type", "policy", "airline", "active"],
    query_type="semantic", query_language="en-us", semantic_configuration_name='my-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:

    print(f"Search Reranked Score: {result['@search.reranker_score']}")
    print(f"Search Score: {result['@search.score']}")
    print(f"id: {result['id']}")
    print(f"policy_type: {result['policy_type']}")
    print(f"policy: {result['policy']}")
    print(f"airline: {result['airline']}")
    print(f"active: {result['active']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


In [ ]:
new_policy = {"id": "e22891ad-5850-42ac-b856-270615ab1f3a",
"policy_type": "checked baggage policy",
"policy": """American Airlines' checked baggage policy states that passengers are allowed to check up to 2 bags on their flights. However, charges will apply for all checked bags except for First Class and Business Class passengers, AAdvantage Executive Platinum, Platinum Pro, and Platinum members, and Oneworld Emerald members. 

For all other passengers, the first checked bag will cost $30 for domestic flights and flights to/from Canada, Mexico, Central America, and the Caribbean, and $60 for transatlantic flights. The second bag will cost $40 for domestic flights and flights to/from Canada, Mexico, Central America, and the Caribbean, and $100 for transatlantic flights. 

Additional checked bags will each cost $150 for all flights except for flights to/from Brazil, where each additional checked bag costs $85. Oversized and overweight baggage fees also apply, with fees ranging from $100 to $450 per bag depending on the weight and size. 

Passengers are encouraged to check their baggage allowance and fees prior to their flights to avoid any unexpected charges at the airport.""",
"airline":  "American Airlines",
"active":  "true"}
contentVector = generate_embeddings(new_policy['policy'])
new_policy['contentVector'] = contentVector


In [ ]:

def add_or_update(new_policy):
    azcs_search_client.merge_or_upload_documents(documents = [new_policy])
add_or_update(new_policy)